# Задача о рюкзаке

**Задача о рюкзаке** — задача, в которой требуется выбрать подмножество предметов так, чтобы суммарная ценность была максимальной, а суммарный вес не превышал заданную грузоподъёмность рюкзака.

Дано: рюкзак вместимостью $m$ кг., $n$ предметов с весами $w=\{w_1, w_2, \ldots, w_n\}$, а также стоимостями $p=\{p_1,p_2,\ldots,p_n\}$. 

Нужно решить, какие предметы взять, чтобы не превышать вместимость рюкзака и при этом получить максимальную суммарную ценность

## Достижимость

Давайте для начала решим задачу попроще: можно ли набрать суммарный вес ровно `j`, используя первые `i` предметов?

Заведем массив `dp`, где `dp[i][j]=1`, если можно собрать рюкзак весом `j`, взяв первые `i` предметов. Иначе - `dp[i][j]=0`

Тогда, для `dp[i][j]` есть два варианта:
- не берем предмет `i`, тогда `dp[i][j]=d[i-1][j]`
- берем предмет `i` - тогда нам нужно, чтобы на шаге раньше можно было набрать вес `j - w[i]`, то есть `dp[i][j] = dp[i - 1][j - w[i]]`

После заполнения `dp` мы знаем какие веса вообще достижимы с данным набором предметов и какие веса не достижимы ни при каких комбинациях

**Пример.** Рассмотрим рюкзак с максимальной вместимостью 14 кг., $n=4$ предмета с весами $w=\{3, 4, 8, 5\}$

In [1]:
W = 14
w = [0] + [3, 4, 8, 5]
n = len(w)

dp = [[0] * (W + 1) for i in range(n)]
dp[0][0] = 1

for i in range(1, n):
  wi = w[i]
  for j in range(W + 1):
    dp[i][j] = dp[i - 1][j]
    if j - wi >= 0 and dp[i - 1][j - wi] == 1:
       dp[i][j] = 1
dp

[[1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
 [1, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
 [1, 0, 0, 1, 1, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0],
 [1, 0, 0, 1, 1, 0, 0, 1, 1, 0, 0, 1, 1, 0, 0],
 [1, 0, 0, 1, 1, 1, 0, 1, 1, 1, 0, 1, 1, 1, 0]]

А еще эту задачу пожно решить с помощью битовых операций. 

Создадим число `dp`. Его бит с индексом `j` равен 1, если сумму `j` можно набрать некоторым подмножеством предметов. Изначально `dp=1`, так как сначала достижима только сумма 0

Далее для каждого веса `wi` будем производить сдвиг:
```python
dp = (dp << wi) | dp
```
Если раньше была достижима сумма `j`, то после сдвига `dp << wi` становится достижима сумма `j + wi`, то есть берём предмет `wi`. Побитовое ИЛИ (знак `|`) объединяет предыдущие достижимые суммы (то есть не берём предмет) и новые (берём предмет)

In [2]:
W = 14
w = [3, 4, 8, 5]
dp = 1
for i in w:
  dp = (dp << i) | dp

В результате в числе `dp` будут выставлены единицы ровно в тех позициях, какие веса можно набрать. Чтобы посмотреть, какие именно суммы от 0 до `W` достижимы, нужно перевести это число в двоичную систему и развернуть её

In [3]:
bin(dp)[2:][::-1][:15]

'100111011101110'

## Поиск максимальной стоимости

Теперь усложним задачу - добавим цены для каждого предмета

Создадим двумерный массив `dp`. `dp[i][j]` будет показывать максимальную стоимость предметов, которые можно положить в рюкзак вместимостью `j`, если можно использовать только первые `i` предметов

Для любых `i,j` возможны два варианта:
- либо мы не берем `i-й` предмет в рюкхак, тогда `dp[i][j]=dp[i-1][j]`, то есть равно максимальной стоимости рюкзака с такой же вместимостью и набором допустимых предметов $\{n_1,n_2,\ldots, n_{i-1}\}$
- либо мы берем предмет `i` и тогда `dp[i][j]=dp[i-1][j-w_i]+p_i`, то есть мы уменьшаем доступную вместимость на $w_i$ и добавляем стоимость предмета $i$

Так как нам нужно максимально укомплектовать рюкзак, то 
$$dp[i][j]=\max(dp[i-1][j];\ dp[i-1][j-w_i]+p_i)$$

Таким образом, стоимость искомого набора будет в `dp[n][m]`

**Пример.** Пусть $m=13,\ n=5,\ w=\{3, 4, 5, 8, 9\},\ p=\{1, 6, 4, 7, 6\}$

Создаем `dp`. Числа в верхней строке - вместимость рюкзака, а `i` - сколько *первых* предметов мы берем
<img src="img/dp16.jpg">

Очевидно, что взяв первые 0 предметов стоимость рюкзака булет равен 0. Перейдем к строке `i=1`: как только вместимость рюкзака становится равна 3 кг. мы можем положить в него первый предмет. При большей вместимости мы ничего в рюкхак докинуть не сможем, поэтому там останется стоимость в 1 единицу

<img src="img/dp17.jpg">

Рассмотрим $i=2$. При вместимости рюкзака до 3 кг. мы не сможем взять ни 1-й предмет, ни 2-й. 

Далее, при вместимости 3 мы сможем взять первый предмет, поэтому стоимость рюкхака на этом этапе стала равна 1

При вместимости рюкзака 4 нам нужно выбрать что выгоднее - оставить первый предмет или отказаться от него и положить второй:
$$dp[2][4]=\max(dp[1][4];\ dp[1][0]+6)=\max(1;6)=6$$
Значит, нам выгоднее взять второй предмет

При вместимостях 5 и 6 ситуация аналогичная

При вместимости 7:
$$dp[2][7]=\max(dp[1][7];\ dp[1][3]+6)=\max(1;7)=7$$
Получается, мы смогли увелить стоимость рюкзака, докинув в него первый предмет, так как вместимость позволяет

<img src="img/dp18.jpg">

Проводя аналогичные рассуждения получаем такой массив `dp`

<img src="img/dp19.jpg">

Значит, максимальная стоимость рюкзака равна 13

**Сложность.** Такое решение работает за $O(n\cdot m)$

In [9]:
n, m = map(int, input().split())
w = [0] + list(map(int, input().split()))
c = [0] + list(map(int, input().split()))

dp = [[0] * (m + 1) for _ in range(n + 1)]

for i in range(1, n + 1):
    for j in range(1, m + 1):
        dp[i][j] = dp[i - 1][j]
        if j - w[i] >= 0:
            dp[i][j] = max(dp[i][j], dp[i - 1][j - w[i]] + c[i])

print(dp[n][m])

13


## Восстановление ответа

Чтобы восстановить ответ нужно пойти в обратном порядке по `i` от ячейки `dp[n][m]`.

Алгоритм:
- если `dp[i][j] == dp[i-1][j]` это означает, что мы не положили `i-й` предмет и нужно запускаться от `dp[i-1][j]`
- если `dp[i][j] != dp[i-1][j]`, значит мы положили в рюкзак `i-й` предмет и дальше нужно запускаться от `dp[i-1][j-w_i]`. Таким образом мы какк бы вытаскиваем из рюкзака вместимости `j` наш предмет и смотрим, что мы могли положить в рюкзак меньшей вместимости

Рассмотрим наш пример, встаем в ячейку `dp[5][13]`:
- `dp[5][13]=dp[4][13]`, значит переходим в `dp[4][13]`
- `dp[4][13] != dp[3][13]`, значит переходим в `dp[3][13-8]=dp[3][5]`. Совершая такой "прыжок" по матрице мы находим предмет, который положили в рюкзак. В данном случае, это четвертый предмет
- `dp[3][5]=dp[2][5]`, значит переходим в `dp[2][5]`
- `dp[2][5] != dp[1][5]`, значит снова прыгаем, а значит мы нашли еще один предмет, который взяли в рюкзак

Итого: в рюкзак вошли 2-й и 4-й предметы, общим весом 13 кг. и стоимостью 12 ед.

<img src="img/dp20.jpg">

In [12]:
x = m
selected_items = []
for i in range(n, 0, -1):
    if dp[i][x] != dp[i - 1][x]:
        selected_items.append(i)
        x -= w[i]
selected_items.reverse()

for item in selected_items:
    print(item, end=' ')

2 4 